In [2]:
pip install scikit-learn==1.0.2

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -U imbalanced-learn==0.9.1

Note: you may need to restart the kernel to use updated packages.


In [65]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
from pyarrow.csv import read_csv
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import sklearn.svm as svm
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from lightgbm import LGBMClassifier

전처리 방법: 주말과 주중의 평균을 각각 이용



In [198]:
X = read_csv('X_model_nnew.csv').to_pandas()
X = X.iloc[:,1:]
y = read_csv('Y_model.csv').to_pandas()
X.fillna(0, inplace=True)
X2 = read_csv('X_model_new.csv').to_pandas()
y = read_csv('Y_model.csv').to_pandas()
for i in range(8):
    X2[f's{i+1}'] = X2[f's{i+1}']**0.1

In [7]:
X2.loc[:,'s1':'s8']

,s1,s2,s3,s4,s5,s6,s7,s8
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.573433
1,1.557500,1.537883,0.000000,1.673219,1.327532,1.658021,1.634383,1.650476
2,0.000000,0.000000,0.000000,0.000000,2.164094,1.907278,2.105476,1.867880
3,0.000000,0.000000,0.000000,0.000000,1.792267,1.771862,0.000000,0.000000
4,1.591121,1.430969,1.641454,1.487399,1.752331,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
799995,0.000000,0.000000,0.000000,0.000000,1.901818,0.000000,1.570016,1.231144
799996,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.036666
799997,2.103128,1.720294,1.923971,2.289883,2.254675,1.767753,1.818735,1.858645
799998,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.228217


In [200]:
X

,gender,age_code,region_code,c_week_mean,c_weekend_mean,t_week_mean,t_weekend_mean,s1,s2,s3,s4,s5,s6,s7,s8
0,1,13,7,0.005882,0.000000,0.005882,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.573433
1,1,5,1,0.211765,0.088235,0.000000,0.000000,1.557500,1.537883,0.000000,1.673219,1.327532,1.658021,1.634383,1.650476
2,2,6,2,0.094118,0.088235,0.011765,0.029412,0.000000,0.000000,0.000000,0.000000,2.164094,1.907278,2.105476,1.867880
3,2,1,1,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.792267,1.771862,0.000000,0.000000
4,2,5,1,0.041176,0.029412,0.005882,0.000000,1.591121,1.430969,1.641454,1.487399,1.752331,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,1,6,16,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.901818,0.000000,1.570016,1.231144
799996,2,11,12,0.005882,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.036666
799997,1,4,7,0.582353,0.411765,0.182353,0.117647,2.103128,1.720294,1.923971,2.289883,2.254675,1.767753,1.818735,1.858645
799998,1,7,1,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.228217


In [199]:
X = pd.concat((X.drop(columns=['s_week_mean', 's_weekend_mean']),X2.loc[:,'s1':'s8']), axis=1)

In [205]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    stratify = y, shuffle = True, 
                                                    random_state = 42, test_size=0.2) 

In [206]:
X_train

,gender,age_code,region_code,c_week_mean,c_weekend_mean,t_week_mean,t_weekend_mean,s1,s2,s3,s4,s5,s6,s7,s8
354819,2,4,15,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.430969,1.405116
559807,1,8,1,0.070588,0.029412,0.000000,0.000000,1.764161,1.116123,1.292392,1.385152,1.426944,1.446126,1.327532,1.449701
147728,1,4,17,0.000000,0.029412,0.000000,0.000000,1.737670,0.000000,1.849768,0.000000,0.000000,0.000000,0.000000,0.000000
259344,2,7,7,0.341176,0.088235,0.070588,0.058824,2.032819,1.838416,2.192771,1.931585,2.075459,2.002718,1.821472,1.939196
373127,2,8,1,1.641176,1.176471,0.923529,0.794118,2.427279,2.345169,2.226661,2.346331,2.367875,2.336545,2.326926,2.263099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94416,1,6,14,2.482353,2.882353,1.464706,2.235294,2.511509,2.241937,2.333979,2.326676,2.249373,2.176724,2.406752,2.172680
254577,2,9,2,0.005882,0.000000,0.000000,0.000000,0.000000,0.000000,1.865701,0.000000,0.000000,0.000000,0.000000,0.000000
457328,2,9,9,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,1.750396,1.595653,0.000000,0.000000,0.000000,0.000000
787208,2,6,15,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.319508,1.071773,0.000000


In [210]:
predict_probas = []
for i in range(11,15):
    for j in range(1,11):
        model = XGBClassifier(n_estimators=150, learning_rate=0.2, max_depth=5, random_state = 42, 
                          predictor='gpu_predictor',  tree_method = 'gpu_hist',gpu_id=0, 
                          scale_pos_weight=i, max_delta_step=j)
    #model = LGBMClassifier(boosting_type='gbdt', n_estimators=150, max_depth=6, learning_rate=0.2, 
    #                       gpu_use_dp=True, gpu_device_id=0, scale_pos_weight=i, boost_from_average=False,
    #                       device_type='cpu', pos_bagging_fraction=0.9, neg_bagging_fraction=1, 
    #                       subsample_freq=500,
    #                       lambdarank_norm=True)
        model.fit(X_train, y_train.values.ravel())
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)
        predict_probas.append(y_pred_proba[:,1])
        print(confusion_matrix(y_test, y_pred))
        acc = accuracy_score(y_test, y_pred)
        print('accuracy_score : ',acc)

        pre = precision_score(y_test, y_pred)
        print('precision_score : ', pre)

        recall = recall_score(y_test, y_pred)
        print('recall_score: ', recall)

        roc = roc_auc_score(y_test, y_pred_proba[:,1])
        print('auc roc score: ', roc)
    
        x=0
        for k in range(len(y_pred)):
            if y_pred[k]==1:
                x+=1
                
        print('scale_pos_weight= ', i)
        print('max_delta_step= ', j)
        print('Expected income= ', (5000*x*pre-400*x))
        print('----------------------------------------')

[[112771  36766]
 [  1505   8958]]
accuracy_score :  0.76080625
precision_score :  0.19591461814364447
recall_score:  0.8561598012042435
auc roc score:  0.8822765808517022
scale_pos_weight=  11
max_delta_step=  1
Expected income=  26500400.0
----------------------------------------
[[113066  36471]
 [  1543   8920]]
accuracy_score :  0.7624125
precision_score :  0.19651472758916966
recall_score:  0.8525279556532543
auc roc score:  0.8821989520246076
scale_pos_weight=  11
max_delta_step=  2
Expected income=  26443600.0
----------------------------------------
[[113007  36530]
 [  1529   8934]]
accuracy_score :  0.76213125
precision_score :  0.19650712651768432
recall_score:  0.853866004014145
auc roc score:  0.8825411264290662
scale_pos_weight=  11
max_delta_step=  3
Expected income=  26484400.0
----------------------------------------
[[113030  36507]
 [  1543   8920]]
accuracy_score :  0.7621875
precision_score :  0.19635899355009134
recall_score:  0.8525279556532543
auc roc score:  0